In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle
import shutil

In [2]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:1' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:',device)

True
학습을 진행하는 기기: cuda:1


In [3]:
len(string.printable)
## train args
weight_dir='./new_label_final data5_44000/'
data_mode = 'label'

In [4]:
##decoding

all_letters = string.printable
n_letters = len(all_letters)
def letterToIndex(letter):
    return all_letters.find(letter)
def linesToTensor(lines):
    char_length = 7000

    tensor_one_hot = torch.zeros(len(lines), char_length, n_letters)
    tensor_label = torch.zeros(len(lines),char_length,1) ##line = batchsize
    
    for b, line in enumerate(lines): 
        len_line = min(char_length,len(line))
        line = line[:len_line]
        for li, letter in enumerate(line):
            tensor_one_hot[b][li + char_length - (len_line)][letterToIndex(letter)] = 1 #뒤로 맞춰줌 batch*character*100
            tensor_label[b][li + char_length - (len_line)][0] =letterToIndex(letter) #batch*character*1

    return tensor_one_hot,tensor_label

In [5]:

class LangModel(nn.Module):
    def __init__(self, preTrained='True', input=100):
        super(LangModel, self).__init__()

        # Language Model
        self.lang = nn.LSTM(input, 128, 3, batch_first=True) 
 
        # Output 
        self.output = nn.Linear(128, 2)
        n = self.output.in_features * self.output.out_features
        self.output.weight.data.normal_(0, math.sqrt(2. / n))
        self.output.bias.data.zero_()

    def forward(self, text):
        text.to(device)
        h0 = ( Variable(torch.zeros(3, text.size(0), 128)).to(device),  Variable(torch.zeros(3, text.size(0), 128)).to(device))

        lang_feature, hn = self.lang(text, h0 )
        lang_feature = lang_feature[:,-1,:]

        pred = self.output(lang_feature)
        return pred,lang_feature


In [192]:
import torch.utils.data as data

class chat_ds(data.Dataset):
    def __init__(self,d_type):
        self.gt_range =  1-0.25
        self.d_type=d_type
        with open('./label.pickle','rb') as f1:  
            self.gt=pickle.load(f1)
        with open('./chat_raw.pkl','rb') as f2:  
            self.text=pickle.load(f2)
        #self.sample=list(map(int,list(self.gt.keys())))
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
        if d_type =='total':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654'] + ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630'] + ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.gt[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)

        begin_pos = 0 
        hl_frames = []
        for it, cur_pos in enumerate(pos_idx):
            if it+1 < len(pos_idx): 
                if((pos_idx[it+1] - cur_pos) > 1):#cur_pos와 cur_pos+1 간격이 1보다 크면, 즉 다른 구간이면
                    begin = int((it+1 - begin_pos) * self.gt_range) + begin_pos
                    hl_frames.extend( pos_idx[begin: it] ) #한구간의 하이라이트 25%만 사용하겠다.
                    begin_pos = it+1


        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[hl_frames] = len(sampling) / float(len(hl_frames))
        self.WeightedSampling = sampling

        self.sum=np.insert(np.cumsum([len(self.gt[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return len(self.WeightedSampling)
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)
            vid = np.where(vid[0]>0)[0][0]
            vframe=index-self.sum[vid]
            game_id=str(self.sample[vid])


            win_text=''
            for idx in range(7): #7 : window size
                if vframe+idx<len(self.text[str(game_id)]):
                    win_text+=self.text[str(game_id)][vframe+idx][:1000]+'\n'

            label=self.gt[str(game_id)][vframe]
            return win_text,label,str(game_id)
        
            
            



In [182]:
class SampleSequentialSampler(sampler.Sampler):
    """Samples elements sequentially, always in the same order.
    Arguments:
        data_source (Dataset): dataset to sample from
        offset (int): offset between the samples
    """

    def __init__(self, data_source, offset=10):
        self.num_samples = len(data_source) 
        self.offset = offset

    def __iter__(self):
        return iter(np.arange(0, self.num_samples, self.offset ))

    def __len__(self):
        return len(np.arange(0, self.num_samples, self.offset ))


In [183]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    if epoch<20:
        lr = 0.01
    else:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return lr

In [184]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [185]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

In [186]:
def clip_gradient(model, clip_norm):
    """Computes a gradient clipping coefficient based on gradient norm."""
    totalnorm = 0
    for pm in model.parameters():
        if pm.requires_grad:
            if pm.grad is not None:
                modulenorm = pm.grad.data.norm()
                totalnorm += modulenorm ** 2

    totalnorm = np.sqrt(totalnorm.cpu())
    norm = clip_norm / max(totalnorm, clip_norm)
    for p in model.parameters():
        if p.requires_grad:
            if p.grad is not None:
                p.grad.mul_(norm)

In [187]:
#game_id='102844212428895431'
train=chat_ds('train')
val=chat_ds('test')

data load fin
data load fin


In [189]:
for i in range(len(train)):
    print(len(train[i][0]))


1304
1421
1624
1549
1624
1624
1227
1016
836
870
893
823
1370
1863
1748
1743
1688
1466
1406
1077
1017
1185
1185
1244
1419
1494
1276
1358
1320
1320
1851
2354
2320
2320
1805
1730
2107
1957
1788
1965
2329
3090
2955
3079
2797
2236
2018
1776
1133
1133
790
646
608
866
869
751
915
839
643
929
703
578
701
652
606
924
862
830
998
991
876
840
721
834
937
1073
1204
1564
1801
2084
2021
1918
1614
1533
1278
1211
1079
1238
1484
1549
1480
1498
1559
1526
1580
1811
1864
1821
2132
2391
2246
1993
1651
1895
2075
2304
1882
1884
2245
2201
2068
1949
1735
1959
1875
1644
1896
2203
2281
2187
1991
2011
1757
1493
1017
840
647
872
859
749
812
995
1023
1275
1055
1129
1257
1304
1583
1665
1382
1715
1660
1829
1697
1249
1198
1283
905
1105
1024
1175
1168
1383
1257
1370
1143
965
1073
1201
1015
994
996
1260
1690
1562
1413
1381
1660
1630
1315
1363
1442
1626
1582
1599
1651
1949
1669
1729
1706
1753
1690
1534
1772
2119
2226
2084
2698
2746
3051
2515
2257
2017
2022
1463
1158
904
1327
1070
932
1099
1111
1252
1335
959
891
939
780
6

504
506
541
652
685
641
562
651
642
722
954
895
1136
1143
935
1254
1175
832
863
738
697
975
675
764
779
771
653
940
682
690
667
709
702
708
439
767
780
690
670
642
670
650
339
317
301
564
554
617
629
644
688
832
564
533
434
499
433
483
323
303
303
287
241
526
513
559
571
708
746
830
606
479
496
857
735
733
730
693
769
1058
929
1030
990
907
938
1013
751
617
523
960
1048
1109
1206
1186
1294
1331
1224
1380
1320
1634
1821
1733
1786
1492
1246
1262
780
646
760
657
910
998
1547
1653
1568
1606
1685
1398
1284
1375
1253
1228
1243
1462
1630
1722
1056
1079
1209
1265
1022
862
972
1225
1509
1381
1584
1603
1611
1522
1222
969
1283
792
802
770
794
872
1214
910
846
753
1016
894
878
554
551
549
586
453
505
441
609
24818
24807
24770
24640
24557
24557
24245
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
72
265
356
472
492
650
776
774
671
625
844
893
1047
1064
1112
1327
1376
1164
1456
1475
1353
1269
1026
1053
1083
748
465
804
836
797
741
683
665
686
386
682
965
100

1844
1989
1707
1931
1896
1917
1830
1964
1831
2028
1458
1557
1693
1706
1644
1611
1685
1855
1752
1753
1915
1728
1709
1752
1689
1792
2196
2078
2262
2316
2054
2147
2034
1655
1664
1696
1788
1875
1995
1909
1995
1951
1697
1735
1780
1530
1562
1512
1464
1488
1437
1266
1511
1687
1704
1901
1961
2145
2513
2822
2586
2599
2644
2610
2674
2450
2388
2809
2692
2760
2871
2810
2893
2480
2099
2221
2176
1935
1728
1613
1529
1587
1890
1711
1905
2079
2106
2171
2121
1855
1908
2091
2116
2155
2217
2554
2333
2346
2242
2215
2597
2990
2598
2570
2568
2644
2622
2288
1899
1934
2035
2014
1730
1845
2073
2332
2575
2691
2564
2932
2826
2747
2518
2254
2351
2595
2310
2254
2057
2193
2583
2654
2668
2508
2445
2289
2029
1524
1349
1428
1435
1327
1435
1349
1343
1079
895
1099
1118
1203
1280
1595
1611
1595
1482
1692
1390
1271
1073
1317
1153
960
1020
1270
1300
1385
1499
1517
1731
1652
1550
1781
1861
1784
1771
1812
1681
1777
1633
2057
2009
2140
2071
2131
2104
2110
1331
1377
1165
979
738
683
560
706
507
455
455
455
293
734
817
868
1219


1390
1186
913
920
805
564
610
764
989
988
999
1102
1171
1081
841
626
585
668
674
767
722
753
789
979
929
889
746
893
938
957
865
954
1056
1027
1168
1099
1279
1183
1274
1181
1167
862
835
752
733
489
522
670
840
898
1023
1067
1028
924
1025
1002
1326
1077
1087
1119
1436
1256
1150
774
787
806
795
655
547
505
691
794
713
883
949
1246
1264
1101
939
1506
1393
1220
949
1140
1319
1907
1448
1799
1844
1939
2046
1924
1787
1911
1587
1708
1658
1358
1251
1202
1082
1076
887
859
1033
1090
707
641
538
753
765
587
644
702
892
1077
1084
1455
1696
1780
1962
1856
1913
1768
1641
1566
1472
1226
1208
1146
1092
942
958
931
903
832
941
1065
1017
943
974
1140
1903
1737
1781
1877
1775
1801
1905
1096
1024
871
935
940
848
863
972
964
913
943
1136
1178
909
931
967
1036
872
1081
1101
1206
1116
1089
1124
1042
669
764
814
917
873
917
1037
1107
1039
870
875
1035
878
879
1075
1046
1076
1093
941
1129
1101
770
921
935
774
912
896
856
923
869
827
871
755
602
712
782
787
905
1059
1254
1364
1192
1141
1162
1047
1187
1192
1117
1

2447
2414
2658
3405
3334
2970
3081
3327
2832
2640
2017
2064
2464
2263
2579
2745
3521
3642
3219
3257
3280
2856
2543
2022
1943
2128
1850
1731
1825
2092
1691
1663
1604
1731
1850
2336
2380
2607
2635
2694
2362
2437
1925
1857
2251
2299
2391
2943
3252
3894
3731
3113
3248
3359
3131
2614
2263
2417
2885
2561
2572
2402
3127
2732
2546
2309
2445
2437
3047
2603
3140
3819
4331
4610
4704
3873
4174
4006
4150
3432
3354
3311
3900
3504
3165
2391
3529
3434
2944
2825
2671
2580
2398
1727
1283
2037
1656
1676
2353
2558
2242
2961
2578
2993
3429
2733
2795
3201
2772
2812
2626
2653
2887
2884
2734
2478
2018
1948
1791
1508
1724
1832
1787
2073
2222
2216
2901
2615
2498
2762
2470
2426
2493
2518
2662
2625
2743
2909
2860
2700
2205
2293
2082
2100
2487
2150
2434
2212
2168
2064
2206
1671
1942
1823
1888
1957
2162
1914
2714
2514
3109
3347
3226
3419
3489
2668
2920
2423
2345
2018
2046
1804
2312
2070
1737
1999
2054
1864
2121
2100
2158
2392
2423
2453
2297
2036
1841
1938
1934
1687
1718
1721
1648
1801
1823
1593
1885
2013
1961
2180


...

1464
1537
1418
1338
1272
1173
1059
1135
1421
1344
1491
1586
1496
1417
1497
1237
1237
1176
1194
1109
1411
1486
1383
1383
1373
1274
1274
1431
1311
1149
1450
1479
1338
1597
1468
1648
1819
1784
1497
1584
1632
1628
1391
1220
1321
1476
1722
1641
1386
1520
1893
1750
1794
1939
1898
2450
2365
2197
2069
2039
1556
1811
1299
1368
1383
2000
2009
2254
2097
2166
1964
1887
1257
1141
1099
1024
1019
1103
1516
1478
1489
1498
1729
1609
1462
1278
1290
1238
1001
809
829
887
571
637
737
844
681
814
942
1239
1529
1456
1462
1427
1267
1590
1364
1001
1014
1192
1218
1235
849
793
841
845
576
600
539
426
382
486
735
813
779
1020
1054
1338
1251
945
853
798
636
590
279
200
195
177
182
101
127
304
336
507
535
777
1057
1014
873
890
751
725
819
548
618
672
621
625
622
519
583
544
537
577
551
602
508
665
738
765
824
873
1063
968
822
726
646
537
708
477
424
435
451
503
485
258
465
488
452
684
677
740
975
854
1000
1074
1138
1274
1208
1043
1187
1364
1403
1236
1414
1444
1751
1757
1484
1337
1256
952
925
586
448
402
817
826
89

839
760
976
992
1002
1017
865
987
911
638
547
635
617
673
537
530
600
702
859
892
857
881
895
865
912
809
750
715
630
807
960
1015
875
900
943
1021
891
807
733
856
898
964
905
912
859
703
607
573
412
555
739
694
744
810
777
822
958
812
856
927
810
838
875
611
510
520
474
602
576
549
644
927
988
1065
937
1000
1125
1077
850
786
701
728
671
510
529
464
416
412
340
423
478
421
527
541
535
587
521
474
499
426
490
527
570
523
634
854
794
863
848
761
795
723
558
609
483
645
654
654
645
597
558
553
417
375
365
366
289
474
456
478
518
661
631
762
659
718
591
651
639
620
640
704
650
707
754
681
867
787
727
876
936
947
1158
1059
1092
1188
1033
1046
982
833
812
806
651
730
660
684
582
705
676
766
754
694
663
703
751
781
716
694
815
801
854
659
631
593
604
530
510
435
429
491
526
511
511
540
490
484
397
370
328
296
250
235
246
254
469
456
490
467
453
432
399
186
195
216
376
470
528
631
778
797
949
854
806
784
721
648
713
559
533
550
576
758
745
678
639
757
819
797
824
791
824
963
1017
1007
1118
111

1505
1492
1666
1508
1404
1184
1226
969
836
889
1015
1057
1075
1106
1164
1231
1145
1127
1145
1050
1058
957
845
821
823
743
1028
1272
1688
1955
2283
2424
2585
2354
2354
2278
2038
1785
1895
1783
2096
1911
1732
1654
1530
1569
1587
1305
1198
1118
1032
1210
918
763
818
715
634
764
625
561
561
616
642
642
681
648
568
807
697
614
729
814
729
1195
1027
920
1131
1520
1301
1759
1860
2124
2296
2231
1883
2014
1775
1342
1007
1011
982
826
884
878
1029
1332
1258
1384
1506
1928
1815
1629
1603
1848
1629
1847
1658
1828
1824
1761
1414
1630
1332
875
641
583
369
693
453
411
545
587
550
654
401
401
694
607
529
607
616
545
643
456
409
505
577
464
648
662
556
813
903
932
932
1038
1017
1017
1028
1088
909
1105
904
813
1040
889
643
643
648
559
972
968
851
1343
1438
1237
1404
1176
953
1129
758
663
908
890
845
845
669
916
916
939
1073
1080
1313
1640
1415
1583
1631
1808
1887
2415
2110
2057
2067
2061
1866
1799
1195
1187
1249
1156
996
955
869
858
1010
944
1148
1263
1174
1252
1207
1161
1219
1081
885
897
821
890
1037
10

14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
24
183
236
317
1067
1437
1462
1507
1730
1813
1863
1309
1114
1463
1820
1520
1612
1760
2180
2295
2074
1909
1955
1978
2008
2085
2196
2224
2346
2359
2250
2123
1727
1464
1421
1324
1383
1390
1706
1502
1423
1601
1554
1678
1720
1327
1363
1397
1266
1347
1062
1285
1200
1100
1097
1039
1037
1339
1249
1457
1528
1543
1432
1275
976
654
795
777
741
825
696
733
1046
754
711
823
738
1116
1094
881
999
992
830
863
570
584
512
333
722
751
771
709
640
632
718
366
443
445
624
916
946
867
832
707
669
708
430
428
426
434
474
572
360
374
324
622
622
827
703
671
656
1053
767
729
510
804
912
978
613
588
828
809
544
421
377
499
491
385
524
520
520
508
639
639
501
355
352
448
500
267
267
283
538
601
546
571
674
697
690
435
344
675
687
631
646
672
881
889
517
713
1006
981
951
816
881
914
625
376
371
498
450
478
532
755
629
668
550
570
495
408
196
284
372
358
312
286
671
639
599
47

KeyboardInterrupt: 

In [142]:
train[13][0]

" COPY BrainSlug THIS BrainSlug GOO BrainSlug TO BrainSlug HELP BrainSlug G2 COPY BrainSlug THIS BrainSlug GOO BrainSlug TO BrainSlug HELP BrainSlug G2 .PogUMA JAN monkaWSKT LETS GOOPromisqPogchamp\n EU <3\n PROMISQmonkaWPROMISQ PogPROMISQ PogPROMISQ LULWPROMISQ PogU9 win 1 lose faker's Neeko Let's go!!!!!why do i hear final boss musicThis should be the finalPROMISQ PogG2monkaWwinner of this series wins worldsLets go Skt🏆🥇Faker win 2-3 AYAYA\n BYE SKT HeyGuysANXIETY CHAT monkaW ANXIETY CHAT monkaWPROMISQ PagChompneekoooneekoooniii!!!!PROMISQ PogEU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LULDM me neeko pics AYAYAratirlL9 uwu ratirlL9 ratirlL9PogPROMISQGo Faker !!MATA LULWPromisq PogpogMATA LULW\n faker is gonna get clappedG2!PROMISQ POGHOLLANDIs this live?\n COPY BrainSlug THIS BrainSlug GOO BrainSlug TO BrainSlug HELP BrainSlug G2 BrainSlug BrainSlug COPY BrainSlug THIS BrainSlug GOO BrainSlug TO BrainSlug HELP BrainSlug G2PROMI

In [141]:
train[14][0]

" EU <3\n PROMISQmonkaWPROMISQ PogPROMISQ PogPROMISQ LULWPROMISQ PogU9 win 1 lose faker's Neeko Let's go!!!!!why do i hear final boss musicThis should be the finalPROMISQ PogG2monkaWwinner of this series wins worldsLets go Skt🏆🥇Faker win 2-3 AYAYA\n BYE SKT HeyGuysANXIETY CHAT monkaW ANXIETY CHAT monkaWPROMISQ PagChompneekoooneekoooniii!!!!PROMISQ PogEU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LUL EU LULDM me neeko pics AYAYAratirlL9 uwu ratirlL9 ratirlL9PogPROMISQGo Faker !!MATA LULWPromisq PogpogMATA LULW\n faker is gonna get clappedG2!PROMISQ POGHOLLANDIs this live?\n COPY BrainSlug THIS BrainSlug GOO BrainSlug TO BrainSlug HELP BrainSlug G2 BrainSlug BrainSlug COPY BrainSlug THIS BrainSlug GOO BrainSlug TO BrainSlug HELP BrainSlug G2PROMISQ OMEGALULCOPY BrainSlug THIS BrainSlug GOO BrainSlug TO BrainSlug HELP BrainSlug G2FREESM LUL FREESM LUL FREESM LUL FREESM LUL FREESM LUL FREESM LUL FREESM LUL FREESM LUL FREESM LUL FREESM LUL FRE

In [193]:
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=44000)
train_loader=torch.utils.data.DataLoader(train,batch_size=128,sampler=sampler1)
# sampler2 =  SampleSequentialSampler(val, 30)
val_loader=torch.utils.data.DataLoader(val,batch_size=128)

In [6]:
###### model load #####
if data_mode == 'label':
    input_val = 1
else:
    inputs_val = 100
model=LangModel(input=input_val).to(device)
criterion = nn.CrossEntropyLoss().to(device)

In [7]:
weight_dir='./new_label_final data5_44000/'

dataset=weight_dir+'26train_best'
checkpoint=torch.load(dataset,map_location='cuda:1')
model.load_state_dict(checkpoint)
model.to(device)

LangModel(
  (lang): LSTM(1, 128, num_layers=3, batch_first=True)
  (output): Linear(in_features=128, out_features=2, bias=True)
)

In [196]:
#train set

if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)


best_f1 = 0
best_losses=1000000
for epoch in range(60):
    lr = adjust_learning_rate(optimizer, epoch)
    train_loss=0
    print(epoch)
    model.train()
    for i, (text,labels,game_id) in enumerate(train_loader):
        if data_mode == 'label':
            inputs=linesToTensor(text)[1]
        else:
            inputs=linesToTensor(text)[0]
        inputs = Variable(inputs).to(device)
        labels = Variable(labels).to(device)
        optimizer.zero_grad()
        output_p,output_f=model(inputs)

        loss=criterion(output_p,labels.long())
        train_loss+=loss
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.) #rnn 계열 gradient 장치
        loss.backward()
        optimizer.step()


    #validation
    model.eval()
    val_loss=0
    acc=0
    gt_sum=0
    
    tp_sum=0
    fp_sum=0
    fn_sum=0
    f1 = 0

    with open(weight_dir+'train_result','a') as f:
        with torch.no_grad():
            for it, (text,labels,game_id) in enumerate(val_loader):
                if data_mode == 'label':
                    inputs=linesToTensor(text)[1]
                else:
                    inputs=linesToTensor(text)[0]
                inputs = Variable(inputs).to(device)
                labels = Variable(labels).to(device)
                output_p,output_f=model(inputs)
                loss=criterion(output_p,labels.long())
                val_loss+=loss
                TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output_p.cpu(),labels.cpu())
                tp_sum += TP
                fp_sum += FP
                fn_sum += FN
                acc=acc+TP+TN
                gt_sum+=len(output_p)
                
        
            val_acc=acc/gt_sum
            if tp_sum>0 and fp_sum>0 and fn_sum>0:
                precision = tp_sum/(tp_sum+fp_sum)
                recall = tp_sum / (tp_sum+fn_sum)
                f1 = (2*precision*recall / (precision + recall)) * 100
            print("epoch {} train_loss : {} , val_loss : {}, val_f1 : {}".format(epoch,train_loss/len(train_loader),val_loss/len(val_loader),f1))
            f.write("epoch {} train_loss : {} , val_loss : {}, val_f1 : {}\n".format(epoch,train_loss/len(train_loader),val_loss/len(val_loader),f1))
            if best_f1<f1:
                best_f1=f1
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train_best"))
                torch.save(model.state_dict(),'{}'.format(weight_dir+"train_best"))
                f.write("epoch {} saved\n".format(epoch))
            else:
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))




0
epoch 0 train_loss : 0.6880388259887695 , val_loss : 0.7010793089866638, val_f1 : 30.359276666932207
1
epoch 1 train_loss : 0.6811571717262268 , val_loss : 0.7401378750801086, val_f1 : 30.826010544815464
2
epoch 2 train_loss : 0.6823403239250183 , val_loss : 0.7154735922813416, val_f1 : 30.72105044711919
3
epoch 23 train_loss : 0.6565105319023132 , val_loss : 0.6956032514572144, val_f1 : 33.86048072486262
24
epoch 24 train_loss : 0.6538561582565308 , val_loss : 0.6796604990959167, val_f1 : 34.136185113329866
25
epoch 25 train_loss : 0.6499280333518982 , val_loss : 0.6291296482086182, val_f1 : 33.979001740497445
26
epoch 26 train_loss : 0.6493849754333496 , val_loss : 0.7114134430885315, val_f1 : 34.521860650828515
27
epoch 27 train_loss : 0.6487985849380493 , val_loss : 0.6295165419578552, val_f1 : 34.07734093476039
28


KeyboardInterrupt: 

In [197]:
#test

test=chat_ds('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=1024)
dataset=weight_dir+'26train_best'
checkpoint=torch.load(dataset,map_location='cuda:1')
model.load_state_dict(checkpoint)
model.to(device)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (text,labels,g) in enumerate(test_loader):
        if data_mode == 'label':
            inputs=linesToTensor(text)[1]
        else:
            inputs=linesToTensor(text)[0]
        inputs = inputs.to(device)
        labels = labels.to(device)
        output_p,output_f=model(inputs)
        
        
        print(output_p)

        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output_p.cpu(),labels.cpu())
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output_p)
        for idx,gi in enumerate(g):
            if gi not in result.keys():
                result[gi]=pred[idx].tolist()
            else:
                result[gi]+=pred[idx].tolist()
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
50 375 548 51 tensor(425) tensor(101)
128 562 309 25 tensor(690) tensor(153)
92 244 636 52 tensor(336) tensor(144)
67 313 619 25 tensor(380) tensor(92)
135 334 515 40 tensor(469) tensor(175)
60 528 388 48 tensor(588) tensor(108)
165 384 377 98 tensor(549) tensor(263)
103 450 421 50 tensor(553) tensor(153)
100 512 381 31 tensor(612) tensor(131)
162 462 356 44 tensor(624) tensor(206)
109 262 518 135 tensor(371) tensor(244)
30 176 762 56 tensor(206) tensor(86)
142 479 327 76 tensor(621) tensor(218)
85 632 288 19 tensor(717) tensor(104)
201 333 437 53 tensor(534) tensor(254)
22 186 775 41 tensor(208) tensor(63)
143 522 325 34 tensor(665) tensor(177)
90 603 282 49 tensor(693) tensor(139)
139 430 380 75 tensor(569) tensor(214)
194 462 315 53 tensor(656) tensor(247)
172 424 344 84 tensor(596) tensor(256)
115 445 399 65 tensor(560) tensor(180)
153 304 513 54 tensor(457) tensor(207)
118 552 325 29 tensor(670) tensor(147)
173 571 227 53 tensor(744) tensor(226)
51 518 437 18 tensor(

In [198]:
def fmeasure2(frames,label): ##measure def for test
    average = [0,0,0,0,0]
    for key in frames.keys():
        TP = len(np.where((np.array(frames[key])==1)&(label[key]==1)==True)[0])
        FP = len(np.where((np.array(frames[key])==1)&(label[key]==0)==True)[0])
        TN = len(np.where((np.array(frames[key])==0)&(label[key]==0)==True)[0])
        FN = len(np.where((np.array(frames[key])==0)&(label[key]==1)==True)[0])
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        accuracy = (TP+TN)/(TP+FN+FP+TN)
        if precision==0 and recall == 0:
            print('!')
        else:
            f1 = (2*precision*recall / (precision + recall))
            print(key)
            print('precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(precision,recall,f1,accuracy))
            average[0]+= precision
            average[1]+= recall
            average[2]+= f1
            average[3]+= accuracy
            average[4]+=1
    print('==precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))

with open('./label.pickle','rb') as f1:  
    gt=pickle.load(f1)
fmeasure2(result,gt)

102844212431058132
precision : 0.1952707856598017, recall : 0.6719160104986877, f1 : 0.30260047281323876, accuracy : 0.5211038961038961
102844341902586509
precision : 0.25598991172761665, recall : 0.8023715415019763, f1 : 0.38814531548757164, accuracy : 0.7006548175865295
102844401152267937
precision : 0.20376712328767124, recall : 0.5920398009950248, f1 : 0.3031847133757962, accuracy : 0.5220620358235037
102844212430927059
precision : 0.20898258478460127, recall : 0.6909090909090909, f1 : 0.32090077410274453, accuracy : 0.49250591638180385
102844412708953395
precision : 0.22016460905349794, recall : 0.38768115942028986, f1 : 0.2808398950131234, accuracy : 0.7334630350194552
102844212429944013
precision : 0.2036673215455141, recall : 0.8382749326145552, f1 : 0.32771338250790305, accuracy : 0.3763440860215054
102844341912679064
precision : 0.22800718132854578, recall : 0.5619469026548672, f1 : 0.3243933588761175, accuracy : 0.727319587628866
102844235753749959
precision : 0.190322580645

In [44]:
len(np.array(result['102844341902586509']))

2138

In [43]:
result.keys()

dict_keys(['102844212431058132', '102844341902586509', '102844401152267937', '102844212430927059', '102844412708953395', '102844212429944013', '102844341912679064', '102844235753749959', '102844341908026005', '102844283023206486', '102844224147717245', '102844412704890154', '102844212430599377', '102844412711443769', '102844235747982779'])

In [ ]:
#test

test=chat_ds('total')
test_loader=torch.utils.data.DataLoader(test,batch_size=1)
dataset=weight_dir+'26train_best'
checkpoint=torch.load(dataset,map_location='cuda:1')
model.load_state_dict(checkpoint)
model.to(device)
model.eval()

result={}
result2={}
result3={}
with torch.no_grad():
    for it, (text,labels,game_id) in enumerate(test_loader):
        if data_mode == 'label':
            inputs=linesToTensor(text)[1]
        else:
            inputs=linesToTensor(text)[0]
        inputs = inputs.to(device)
        labels = labels.to(device)
        output_p,output_f=model(inputs)
#         output=(output)
        _, pred = output_p.topk(1, 1, True, True)
        pred = pred.view(-1,1)
        if game_id[0] not in result.keys():
            print(game_id[0])
            result[game_id[0]]=[int(pred[0][0])]
            result2[game_id[0]]=[(output_f[0]).tolist()]
            result3[game_id[0]]=[(output_p[0]).tolist()]
            
        else:
            result[game_id[0]].append(int(pred[0][0]))
            result2[game_id[0]]+=[(output_f[0]).tolist()]
            result3[game_id[0]]+=[(output_p[0]).tolist()]

        
with open(weight_dir+'chat_bin.json','a') as f:
    json.dump(result,f)
with open(weight_dir+'chat_prob.json','a') as f:
    json.dump(result3,f)
with open(weight_dir+'chat_feature.json','a') as f:
    json.dump(result2,f)


data load fin
102844412722519367
102844212429550795
102844401151219358
102844401154430631
102844412717014335
102844401153971877
102844224148503678
102844412722847048
102844401152857762
102844412707380528
102844212431516886
102844283027925085
102844412716227901
102844412710001974
102844294670878922
102844294670551241
102844283023599703
102844412704496937
102844235751783874
102844401152071328
102844412709674293
102844401153447587
102844224148896895
102844235746868664
102979081290790284
102844283027531868
102844212431975640
102844401155937960
102844212429092040
102844341906649746
102844412706987311
102844412721339716
102844212430402768
102844341905011343


In [37]:
with open(weight_dir+'chat_bin_train.json','a') as f:
    json.dump(result,f)
with open(weight_dir+'chat_prob_train.json','a') as f:
    json.dump(result3,f)
with open(weight_dir+'chat_feature_train.json','a') as f:
    json.dump(result2,f)

In [128]:
x=torch.tensor([[ 0.0124, -0.0438, -0.0314, -0.0224,  0.0108, -0.0189,  0.0026,  0.0232,
         -0.0182, -0.0174,  0.0452,  0.0375, -0.0213,  0.0411, -0.0517,  0.0155,
          0.0682, -0.0051, -0.0309, -0.0049,  0.0018, -0.0063,  0.0157,  0.0178,
          0.0045,  0.0187, -0.0101,  0.0066,  0.0157, -0.0387,  0.0258, -0.0338,
          0.0075,  0.0497,  0.0151,  0.0546,  0.0097, -0.0002,  0.0450,  0.0119,
         -0.0144,  0.0214,  0.0165,  0.0134,  0.0420,  0.0768, -0.0200,  0.0237,
          0.0432,  0.0226,  0.0176,  0.1564,  0.0149, -0.0334,  0.0200,  0.0510,
          0.0013, -0.0187, -0.0430,  0.0499,  0.0285,  0.0552, -0.0237,  0.0297,
          0.0055, -0.0049, -0.0273,  0.0123, -0.0218, -0.0252, -0.0567, -0.0266,
          0.0108, -0.0038,  0.0411,  0.0293,  0.0558, -0.0290, -0.0112, -0.0141,
          0.0392, -0.0739,  0.0045, -0.0106, -0.0030,  0.0166,  0.0260,  0.0099,
          0.0313,  0.0104,  0.0105,  0.0058,  0.0141, -0.0126, -0.0301,  0.0465,
         -0.0182, -0.0304, -0.0034,  0.0460, -0.0364, -0.0233,  0.0174, -0.0238,
          0.0647,  0.0091, -0.0461,  0.0063,  0.0016, -0.0056,  0.0563,  0.0176,
         -0.0710,  0.0373, -0.0593, -0.0405,  0.0052, -0.0136,  0.0320,  0.0190,
          0.0060, -0.0392, -0.0333, -0.0166,  0.0437,  0.0176, -0.0003,  0.0078]],
       device='cuda:1')

In [169]:
a=[x[0].tolist()]

In [171]:
a.append(x[0].tolist())

In [34]:
result['102844224148503678']

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [27]:
len(result['102844235753356742'])

1654

In [17]:
x['102844235753356742']

NameError: name 'x' is not defined

In [24]:
len(result.keys())

60

In [25]:
            sample =[102844235753356742,102844294666422466, 102844341906256529, 102979081290790284, 102844212430271695, 102844401155937960, 102844412704496937, 102844412722519367, 102844235749031358, 102844412709674293, 102844341902586509, 102844235750997440, 102844212431975640, 102844294667995333, 102844341912220311, 102844235747261881, 102844294670551241, 102844412707708209, 102844212430599377, 102844224145685626, 102844401152857762, 102844341908026005, 102844341909598870, 102844283023599703, 102844235753749959, 102844294670026952, 102844212430075086, 102844283027531868, 102844212430927059, 102844212429419722, 102844283025696858, 102844224146472059, 102844412712164667, 102844401153971877, 102844235748310460, 102844412711116088, 102844235752111555, 102844283020453971, 102844294666881219, 102844401152267937, 102844212429944013, 102844294671796427, 102844401151874719, 102844412723567946, 102844294669568199, 102844412709346612, 102844212431779031, 102844212430402768, 102844412711443769, 102844283027925085, 102844235746868664, 102844283023206486, 102844401154168486, 102844212429550795, 102844341907370644, 102844412721339716, 102844294667405508, 102844294674876621, 102844212429288649, 102844401154430631]


In [26]:
len(sample)

60

In [50]:
with open(weight_dir+'chat_128_test.json','rb') as f:
    r=json.load(f)

In [61]:
r['102844412717014335'][:10]

[[0.03537170588970184,
  -0.043081168085336685,
  -0.04096755012869835,
  -0.015679210424423218,
  -0.000730052764993161,
  -0.039760321378707886,
  -0.01698194444179535,
  0.03368096053600311,
  -0.030482346192002296,
  -0.05684709921479225,
  0.07552143186330795,
  0.05213841795921326,
  -0.06367486715316772,
  0.03364301472902298,
  -0.09859336912631989,
  0.020916694775223732,
  0.0760808065533638,
  -0.013417842797935009,
  -0.05708283558487892,
  -0.0011524142464622855,
  0.0006507260841317475,
  -0.011696960777044296,
  0.024534426629543304,
  0.024831000715494156,
  0.013955054804682732,
  0.010192235000431538,
  -0.019946249201893806,
  0.008109033107757568,
  0.005184138659387827,
  -0.05684118717908859,
  0.03394879773259163,
  -0.04553905874490738,
  0.009822734631597996,
  0.05076994746923447,
  0.01525385957211256,
  0.05976983159780502,
  0.03937987610697746,
  0.025555623695254326,
  0.05062054097652435,
  0.0014553380897268653,
  -0.033140629529953,
  0.045567929744720

In [54]:
r['102844412710001974'][:10]

[[0.021551288664340973,
  -0.04384525120258331,
  -0.03462623432278633,
  -0.02016858384013176,
  0.006743958685547113,
  -0.028185468167066574,
  -0.004008247051388025,
  0.026203304529190063,
  -0.024540366604924202,
  -0.03245027735829353,
  0.055668849498033524,
  0.041380468755960464,
  -0.036310501396656036,
  0.03796372935175896,
  -0.06945107877254486,
  0.018607456237077713,
  0.07178831845521927,
  -0.008162139914929867,
  -0.040763359516859055,
  -0.0024862613063305616,
  0.00035980346729047596,
  -0.008969173766672611,
  0.020026447251439095,
  0.020622877404093742,
  0.007058870047330856,
  0.014689539559185505,
  -0.012295215390622616,
  0.008632087148725986,
  0.011394845321774483,
  -0.04461101070046425,
  0.03048529289662838,
  -0.04071923717856407,
  0.008205780759453773,
  0.049318742007017136,
  0.015584717504680157,
  0.057505540549755096,
  0.021161774173378944,
  0.01125938631594181,
  0.04544339329004288,
  0.0056217750534415245,
  -0.023245401680469513,
  0.031

In [38]:
len(result.keys())

70

In [30]:
data = chat_ds('total')

data load fin


In [200]:
with open(weight_dir+'chat_feature.pickle','wb') as f:
    pickle.dump(result2,f)

In [201]:
with open(weight_dir+'chat_feature.pickle','rb') as f:
    xx=pickle.load(f)

In [203]:
xx.

dict_keys(['102844412722519367', '102844212429550795', '102844401151219358', '102844401154430631', '102844412717014335', '102844401153971877', '102844224148503678', '102844412722847048', '102844401152857762', '102844412707380528', '102844212431516886', '102844283027925085', '102844412716227901', '102844412710001974', '102844294670878922', '102844294670551241', '102844283023599703', '102844412704496937', '102844235751783874', '102844401152071328', '102844412709674293', '102844401153447587', '102844224148896895', '102844235746868664', '102979081290790284', '102844283027531868', '102844212431975640', '102844401155937960', '102844212429092040', '102844341906649746', '102844412706987311', '102844412721339716', '102844212430402768', '102844341905011343', '102844235753356742', '102844235750997440', '102844412709346612', '102844412705217835', '102844235752963525', '102844412712164667', '102844412705545516', '102844341912220311', '102844341907370644', '102844235749424575', '102844212429419722',